# Проектный практикум 3. Учебная задача
# Часть 2

**Краткое описание:**

Создать нейронную сеть, способную генерировать текстовые отзывы о различных местах на основе определенных входных параметров, таких как категория места, средний рейтинг и ключевые слова.

Данные: https://github.com/yandex/geo-reviews-dataset-2023

**Описание датасета**
- 500 000 уникальных отзывов
- Только отзывы на организации в России
- Доступны на Яндекс Картах
- Опубликованы с января по июль 2023 года
- Датасет не содержит коротких односложных отзывов
- Тексты очищены от персональных данных (номеров телефонов, адресов почты)


**Состав датасета**

Датасет в формате tskv содержит следующую информацию:

* Адрес организации (address)
* Название организации (name_ru)
* Список рубрик, к которым относится организация (rubrics)
* Оценка пользователя от 0 до 5 (rating)
* Текст отзыва (text)

## Импорт необходимых библиотек

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset

## Загрузка и подготовка данных для обучения

In [ ]:
# Загрузка данных
df = pd.read_csv('/content/drive/MyDrive/Alex/dataset_preprocessed.csv')

In [ ]:
# Проверка на пустые значения
print("Количество пустых значений в 'text':", df['text'].isna().sum())

# Удаление строк с пустыми значениями в 'text'
df = df.dropna(subset=['text'])

Количество пустых значений в 'text': 0


In [ ]:
df['text'] = (
    "Категория: " + df['rubrics'] + "; " +
    "Рейтинг: " + df['average_rating'].astype(str) + "; " +
    "Ключевые слова: " + df['kw_by_rub'] + " -> " +
    "Отзыв: " + df['text']
)

In [ ]:
# Проверка и очистка данных
print("Количество пустых значений в 'text':", df['text'].isna().sum())
df = df.dropna(subset=['text'])
df['text'] = df['text'].astype(str)

Количество пустых значений в 'text': 114


<ipython-input-7-85a1cce106dd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].astype(str)


In [ ]:
# df = df[:100000]

In [ ]:
# Конвертация в датасет
dataset = Dataset.from_pandas(df[['text']])

## Токенизация и загрузка модели

In [ ]:
# Загрузка модели и токенизатора
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [ ]:
# Токенизация
def tokenize_function(examples):
    tokenized_inputs = tokenizer(
        examples["text"],
        truncation=True,
        padding='max_length',
        max_length=512,
    )
    # Add labels that are the same as input_ids
    tokenized_inputs['labels'] = tokenized_inputs['input_ids'].copy()
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/499886 [00:00<?, ? examples/s]

## Дообучение модели

In [ ]:
# Настройка обучения
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=8,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    prediction_loss_only=True,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

<ipython-input-12-442cbe4bdf6f>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


api_key

 1e510ed8d84f1e8402ece3f78c828f81d84ebe9b

In [ ]:
# Обучение
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
100,2.099300
200,0.548700
300,0.512300
400,0.492400
500,0.486400
600,0.478300
700,0.478900
800,0.474600
900,0.466900
1000,0.472000


TrainOutput(global_step=5856, training_loss=0.479511170113673, metrics={'train_runtime': 12726.534, 'train_samples_per_second': 117.837, 'train_steps_per_second': 0.46, 'total_flos': 3.917028851712e+17, 'train_loss': 0.479511170113673, 'epoch': 2.9988477787735244})

In [ ]:
# Сохранение модели
model.save_pretrained('/content/drive/MyDrive/Alex/model')
tokenizer.save_pretrained('/content/drive/MyDrive/Alex/model')

('/content/drive/MyDrive/Emil/model/tokenizer_config.json',
 '/content/drive/MyDrive/Emil/model/special_tokens_map.json',
 '/content/drive/MyDrive/Emil/model/vocab.json',
 '/content/drive/MyDrive/Emil/model/merges.txt',
 '/content/drive/MyDrive/Emil/model/added_tokens.json')

## Генерация нового отзыва

In [ ]:
# Загрузка дообученной модели
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/Alex/model')
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/Alex/model')

In [ ]:
# Ввод новых параметров
input_text = "Категория: музей; Рейтинг: 5.0; Ключевые слова: интересные экспозиции, богатая история -> Отзыв:"


In [ ]:
# Генерация отзыва
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(model.device)
output = model.generate(
    input_ids,
    max_length=150,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    do_sample=True,
    top_p=0.95,
    top_k=60,
    temperature=0.9,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
generated_text

'Категория: музей; Рейтинг: 5.0; Ключевые слова: интересные экспозиции, богатая история -> Отзыв: в москве бываю крайне редко и знаю о чём говорю но в этом году посетили москву. музей очень большой с огромным количеством экспонатов и потрясающих витражей. в основном в тц фуд-корт есть множество прекрасных залов. есть зал с панорамными окнами который мы специально выбрали для посещения. внутри очень уютно. это одно из немногих мест с большим количеством зеркальных поверхностей. с первого же взгляда видно что коллекция была собрана в едином и очень красивом здании. но и это очень здорово. экспозиция впечатляет как и само здание внутри и снаружи. у нас была экскурсия'